In [110]:
import pandas as pd
import numpy as np
import sqlite3
import datetime
import sklearn
import sqlalchemy as sa

In [99]:
from sklearn.model_selection import TimeSeriesSplit

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc

import operator

In [100]:
!ls ../

HSE_final_presentation.pdf  data  notebooks


In [101]:
# создание коннекта к БД и крурсора для запросов
conn = sqlite3.connect('../data/input/user_info_HSE_hashed.sqlite')
c = conn.cursor()

In [102]:
# пример получения перечня таблиц (в данном случае она будет 1)
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print ('Перечень таблиц: {0}\r\n\r\n'.format(c.fetchall()))

# перечень столбцов
c.execute("SELECT * FROM rsa_event_log LIMIT 1;")
cols = [descr[0] for descr in c.description]
print ('Столбцы в таблице: {0}'.format(cols))

Перечень таблиц: [('rsa_event_log',)]


Столбцы в таблице: ['cdf_s_123', 'cdf_s_124', 'cdf_s_127', 'cdf_s_135', 'cdf_s_130', 'cdf_s_129', 'cdf_s_134', 'cdf_s_128', 'cdf_s_138', 'cdf_s_126', 'cdf_s_133', 'cdf_s_136', 'cdf_s_137', 'cdf_s_140', 'cdf_s_178_hashed', 'cdf_s_19', 'cdf_s_20', 'cdf_s_218', 'cdf_s_294', 'cdf_s_299', 'amount_original', 'channel_indicator_desc', 'data_i_118', 'data_i_119', 'data_i_120', 'data_i_154', 'data_s_65', 'event_description', 'event_id', 'event_time', 'ext_acct_number_hashed', 'hardwareid', 'short_date', 'user_acct_number_hashed', 'user_agent_string_hash', 'browser_plugins_hash', 'screen_hash', 'user_id', 'ip_address', 'ip_country', 'ip_region', 'ip_city', 'ip_isp']


In [103]:
# Пример форомирования DataFrame из БД с заполнением типа полей

def transform_cols (df, dict_col_types = None):
    # Расширяйте для необходимых столбцов и их явной типизации
    if dict_col_types is None:
        dict_col_types = {
        'amount_original':(float, 0.0),
        'cdf_s_126':(str, u'null'),
        'cdf_s_138':(str, u'null'),
        'channel_indicator':(str, u'null'),
        'event_description':(str, u'null'),
        'cdf_s_294':(int, 0),
        'cdf_s_140':(float, 0.0),
        'data_i_120':(int, 0),
        'cdf_s_218':(str, u'null'),
        'data_s_65':(int, 0),
        'cdf_s_127':(int, 30),
        'cdf_s_135':(int, 30),
        'cdf_s_130':(int, 30),
        'cdf_s_129':(int, 30),
        'cdf_s_134':(int, 30),
        'data_i_154':(float, np.nan),
        'cdf_s_133':(int, 30),
        'cdf_s_20':(str, u'null'),
        'cdf_s_299':(str, u'null'),
        }
                
    if df.shape[0] > 0:
        df.replace(u'null', np.nan, inplace=True)

        for i in dict_col_types:
            if i in df.columns:
                df[i] = df[i].fillna(dict_col_types[i][1]).astype(dict_col_types[i][0])
    
    return df
    

def df_from_sql (sql, parse_dates = ['event_time','cdf_s_19', 'cdf_s_123','cdf_s_124'], dict_col_types = None, chunk_size = None, engine=engine):
    """
    Функция вощвращающая df в результате переданного sql-запроса
    Также поддерживается приведение столбцов к заданным типам 
    и чтение по блокам
    """
     
    if chunk_size is not None:
        df_iter =  [transform_cols(chunk, dict_col_types) for chunk in pd.read_sql_query(sql, engine, parse_dates = parse_dates, chunksize=chunk_size)]
    else:
        df_iter = transform_cols(pd.read_sql_query(sql, engine, parse_dates = parse_dates), dict_col_types)
        #df_iter = pd.read_sql_query(sql, engine, con=conn,  parse_dates = parse_dates)
    
    return df_iter

In [104]:
# создание индексов для ускорения поиска
# описанные здесь уже созданы в предоставленной БД, при необходимотси можете расширить набор
cols_to_index = ['user_id', 'short_date',  'channel_indicator_desc', 'event_description']

for col in cols_to_index:
    c.execute('CREATE INDEX {ix} on {tn}({cn})'\
            .format(ix='indx_new_{0}'.format(col), tn='rsa_event_log', cn=col)) # add new index indx_new_{0}

c.execute('CREATE INDEX date_user_event on rsa_event_log(short_date, user_id, event_description);')   
    
conn.commit()

OperationalError: index indx_new_user_id already exists

In [105]:
#  Общий срез по данным
c.execute("select COUNT(*) from rsa_event_log;")
print ('Общее число транзакций: {0}'.format(c.fetchall()))


# Число уникальных пользователей
c.execute("select COUNT(DISTINCT user_id) from rsa_event_log;")
print ("Уникальных пользователей: {0}".format(c.fetchall()))

# Временной диапазон и количество клиентов
c.execute("select min(short_date), max(short_date) from rsa_event_log;")
print('Диапазон дат: {0}'.format(c.fetchall()))

Общее число транзакций: [(11832149,)]
Уникальных пользователей: [(85764,)]
Диапазон дат: [(20171007, 20171205)]


In [106]:
all_users = df_from_sql('select DISTINCT user_id from rsa_event_log;',
                        engine=conn,
                        parse_dates=None)

In [107]:
sample_users = all_users.sample(frac=0.05).user_id.astype(str).tolist()

In [108]:
start_date = 20171129
end_date = 20171205

In [77]:
data = df_from_sql('select * from rsa_event_log where short_date >= 20171129 and short_date <=  20171205;',engine=conn,
                        parse_dates=None)

In [79]:
data.to_csv('test_data.csv', sep='\t')

0          20171129
1          20171129
2          20171129
3          20171129
4          20171129
5          20171129
6          20171129
7          20171129
8          20171129
9          20171129
10         20171129
11         20171129
12         20171129
13         20171129
14         20171129
15         20171129
16         20171129
17         20171129
18         20171129
19         20171129
20         20171129
21         20171129
22         20171129
23         20171129
24         20171129
25         20171129
26         20171129
27         20171129
28         20171129
29         20171129
             ...   
1996443    20171205
1996444    20171205
1996445    20171205
1996446    20171205
1996447    20171205
1996448    20171205
1996449    20171205
1996450    20171205
1996451    20171205
1996452    20171205
1996453    20171205
1996454    20171205
1996455    20171205
1996456    20171205
1996457    20171205
1996458    20171205
1996459    20171205
1996460    20171205
1996461    20171205


In [85]:
data.to_csv('test_data.csv', sep=',')

In [94]:
data

,cdf_s_123,cdf_s_124,cdf_s_127,cdf_s_135,cdf_s_130,cdf_s_129,cdf_s_134,cdf_s_128,cdf_s_138,cdf_s_126,...,user_acct_number_hashed,user_agent_string_hash,browser_plugins_hash,screen_hash,user_id,ip_address,ip_country,ip_region,ip_city,ip_isp
0,None,None,30,30,30,30,5,NaN,null,null,...,None,None,None,None,10194239,94.25.171.188,ru,48,Moscow,6095671.0
1,None,None,30,30,30,30,5,NaN,null,null,...,None,None,None,None,10194239,94.25.171.188,ru,48,Moscow,6095671.0
2,None,None,30,30,30,30,0,NaN,null,null,...,None,None,None,None,1025801,85.172.82.254,ru,24,Elista,7042071.0
3,None,None,30,30,30,30,0,NaN,null,null,...,None,None,None,None,1025801,85.172.82.254,ru,24,Elista,7042071.0
4,None,None,30,30,30,30,5,NaN,НЕТ,НЕТ,...,ab384e61bdf71c7d28e7f3574ba471da93ae8bcb,None,None,None,10273512,93.188.122.91,ru,48,Moscow,2225362.0
5,None,None,30,30,30,30,0,NaN,НЕТ,ДА,...,45ac3cab944dbc9ca619063935c55efbb2578f8b,None,None,None,1029836,213.87.133.211,ru,48,Moscow,7255229.0
6,None,None,30,2,30,30,0,NaN,НЕТ,ДА,...,79d9e7c08fc8a4ab12db612547341c93b8a1cae2,None,None,None,10872185,31.23.169.60,ru,61,Rostov-na-donu,7042071.0
7,None,None,30,2,30,30,0,NaN,НЕТ,ДА,...,3f6593c23eda6cd4a39dca74f7f7818577fd10b6,None,None,None,10872185,31.23.169.60,ru,61,Rostov-na-donu,7042071.0
8,None,None,30,30,30,30,5,NaN,null,null,...,None,None,None,None,10958673,79.134.74.26,ru,48,Moscow,7377135.0
9,None,None,30,30,30,30,5,NaN,НЕТ,ДА,...,66273fd45f91ebe919493ce04a5632ebf82b42ca,None,None,None,10958673,79.134.74.26,ru,48,Moscow,7377135.0


In [86]:
d = pd.read_csv('../data/test_data/test_data.csv')

In [92]:
d.columns

Index(['Unnamed: 0', 'cdf_s_123', 'cdf_s_124', 'cdf_s_127', 'cdf_s_135',
       'cdf_s_130', 'cdf_s_129', 'cdf_s_134', 'cdf_s_128', 'cdf_s_138',
       'cdf_s_126', 'cdf_s_133', 'cdf_s_136', 'cdf_s_137', 'cdf_s_140',
       'cdf_s_178_hashed', 'cdf_s_19', 'cdf_s_20', 'cdf_s_218', 'cdf_s_294',
       'cdf_s_299', 'amount_original', 'channel_indicator_desc', 'data_i_118',
       'data_i_119', 'data_i_120', 'data_i_154', 'data_s_65',
       'event_description', 'event_id', 'event_time', 'ext_acct_number_hashed',
       'hardwareid', 'short_date', 'user_acct_number_hashed',
       'user_agent_string_hash', 'browser_plugins_hash', 'screen_hash',
       'user_id', 'ip_address', 'ip_country', 'ip_region', 'ip_city',
       'ip_isp'],
      dtype='object')

In [93]:
d.drop('Unnamed: 0',axis = 1)

,cdf_s_123,cdf_s_124,cdf_s_127,cdf_s_135,cdf_s_130,cdf_s_129,cdf_s_134,cdf_s_128,cdf_s_138,cdf_s_126,...,user_acct_number_hashed,user_agent_string_hash,browser_plugins_hash,screen_hash,user_id,ip_address,ip_country,ip_region,ip_city,ip_isp
0,NaN,NaN,30,30,30,30,5,NaN,null,null,...,NaN,NaN,NaN,NaN,10194239,94.25.171.188,ru,48,Moscow,6095671.0
1,NaN,NaN,30,30,30,30,5,NaN,null,null,...,NaN,NaN,NaN,NaN,10194239,94.25.171.188,ru,48,Moscow,6095671.0
2,NaN,NaN,30,30,30,30,0,NaN,null,null,...,NaN,NaN,NaN,NaN,1025801,85.172.82.254,ru,24,Elista,7042071.0
3,NaN,NaN,30,30,30,30,0,NaN,null,null,...,NaN,NaN,NaN,NaN,1025801,85.172.82.254,ru,24,Elista,7042071.0
4,NaN,NaN,30,30,30,30,5,NaN,НЕТ,НЕТ,...,ab384e61bdf71c7d28e7f3574ba471da93ae8bcb,NaN,NaN,NaN,10273512,93.188.122.91,ru,48,Moscow,2225362.0
5,NaN,NaN,30,30,30,30,0,NaN,НЕТ,ДА,...,45ac3cab944dbc9ca619063935c55efbb2578f8b,NaN,NaN,NaN,1029836,213.87.133.211,ru,48,Moscow,7255229.0
6,NaN,NaN,30,2,30,30,0,NaN,НЕТ,ДА,...,79d9e7c08fc8a4ab12db612547341c93b8a1cae2,NaN,NaN,NaN,10872185,31.23.169.60,ru,61,Rostov-na-donu,7042071.0
7,NaN,NaN,30,2,30,30,0,NaN,НЕТ,ДА,...,3f6593c23eda6cd4a39dca74f7f7818577fd10b6,NaN,NaN,NaN,10872185,31.23.169.60,ru,61,Rostov-na-donu,7042071.0
8,NaN,NaN,30,30,30,30,5,NaN,null,null,...,NaN,NaN,NaN,NaN,10958673,79.134.74.26,ru,48,Moscow,7377135.0
9,NaN,NaN,30,30,30,30,5,NaN,НЕТ,ДА,...,66273fd45f91ebe919493ce04a5632ebf82b42ca,NaN,NaN,NaN,10958673,79.134.74.26,ru,48,Moscow,7377135.0


In [112]:
import scipy
scipy.sparse.csr_matrix(data.values)

TypeError: no supported conversion for types: (dtype('O'),)